## GAN

### Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses, callbacks
from tensorflow.keras.models import save_model, load_model, Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.utils import plot_model, Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random as r
from tqdm import tqdm # progress bar
from IPython.display import clear_output
import seaborn as sns

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPUs detected: {len(gpus)}")
        print(f"GPUs: {gpus}")
    except RuntimeError as e:
        print(e)
else:
    print("No GPUs detected")

tf.config.optimizer.set_jit(True)  # Active JIT (XLA) globalement


### Constants

In [ ]:
# Paths
ANNOTDIR = 'datasets/annotations_trainval2014'
DATADIR = 'datasets/train2014'
INSTANCEFILE = '{}/annotations/instances_{}.json'.format(ANNOTDIR, DATADIR)


# Constants
IMAGE_SHAPE = (64, 64, 3)
BATCH_SIZE = 32*4
EPOCHS = 30
OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=1e-4)
IDS = [ f'datasets/Humans_Face/{element}' for element in os.listdir('datasets/Humans_Face') ]
IDS += [ f'datasets/celeba/{element}' for element in os.listdir('datasets/celeba') ]
r.shuffle(IDS)
PATIENCE = 2


#PATH_DATASETS = ['stable-diffusion-face-dataset/512/man','stable-diffusion-face-dataset/512/woman']
TRAIN_SPLIT = 0.8
VALIDATION_SPLIT = 0.15
TEST_SPLIT = 0.05

# Configuration de l'augmentation
DATA_GEN = ImageDataGenerator(
    horizontal_flip=True,
)
BASE_GEN = ImageDataGenerator()

def inference():
    pass

def plot_inference(image=None):
    result = inference(for_plot=True, image=image)
    fig, axes = plt.subplots(1, len(result), figsize=(20, 5))
    for i, res in enumerate(result):
        step, img = res
        img = (img + 1) / 2 # Convertion de -1, 1 à 0, 1
        axes[i].imshow(np.clip(img, 0, 1))
        axes[i].set_title(f'{int(step)}')
        axes[i].axis('off')
    plt.show()

def array_stats(array, plot=False):
    print(f'Shape: {array.shape} \n \
        Mean: {np.mean(array)} \n \
        Min: {np.min(array)} \n \
        Max: {np.max(array)} \n \
        Std: {np.std(array)}')
    if plot:
        sns.histplot(array.flatten())
        plt.show()


### Dataset

In [ ]:
# Face Dataset
class DatasetGeneratorFace(Sequence):
    def _getsplit(self, ensemble):
        if ensemble == 'train':
            start = 0
            stop = int(TRAIN_SPLIT * len(IDS))
        elif ensemble == 'val':
            start = int(TRAIN_SPLIT * len(IDS))
            stop = int((TRAIN_SPLIT + VALIDATION_SPLIT) * len(IDS))
        elif ensemble == 'test':
            start = int((TRAIN_SPLIT + VALIDATION_SPLIT) * len(IDS))
            stop = len(IDS)
        return start, stop
    
    def __init__(self, ensemble, **kwargs):
        super().__init__(**kwargs)
        self.ensemble = ensemble
        
        # Créer une liste de tous les IDs d'images
        start, stop = self._getsplit(ensemble)
        self.ids = IDS[start:stop]
    
    def __len__(self):
        return int(np.ceil(len(self.ids) / BATCH_SIZE))
    
    def __getitem__(self, index):
        batch_ids = self.ids[index * BATCH_SIZE : (index + 1) * BATCH_SIZE]
        batch_images = []
        for id in batch_ids:
            # Charger l'image
            image = load_img(id, target_size=(IMAGE_SHAPE[0], IMAGE_SHAPE[1]), color_mode='rgb')
            image = img_to_array(image)
            image = (image / 255.0) * 2 - 1 # Normalisation entre -1 et 1
            if self.ensemble == 'train':
                image = DATA_GEN.random_transform(image)
            else :
                image = BASE_GEN.random_transform(image)
            batch_images.append(image)
        
        batch_images = np.array(batch_images, dtype='float64')

        return (batch_images, batch_images) # On renvoie l'image d'entrée et la cible (auto-encodeur)

    def on_epoch_end(self):
        if self.ensemble == 'train': # Sert à rien de shuffle les données de validation et de test
            np.random.shuffle(self.ids)

TrainGen = DatasetGeneratorFace('train')
ValGen = DatasetGeneratorFace('val')
TestGen = DatasetGeneratorFace('test')
print(f"Train: {len(TrainGen)} \n Val: {len(ValGen)} \n Test: {len(TestGen)}")

# Visualisation des données
def plot_data_gen(gen, n=5):
    fig, axes = plt.subplots(1, n, figsize=(20, 5))
    for i in range(n):
        image = gen[i][0][0]
        image = (image + 1) / 2 # Convertion de -1, 1 à 0, 1
        axes[i].imshow(np.clip(image, 0, 1))
        axes[i].axis('off')
    plt.show()

plot_data_gen(TrainGen, n=5)
plot_data_gen(ValGen, n=5)
plot_data_gen(TestGen, n=5)
    

### Model

In [ ]:
def build_GeneratorModel():
    noise_dim = 100
    noise_input = layers.Input(shape=(noise_dim,))

    x = layers.Dense(4*4*1024, use_bias=False)(noise_input)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Reshape((4, 4, 1024))(x)

    x = layers.Conv2DTranspose(512, kernel_size=5, strides=2, padding='same', use_bias=False)(x)  # 8x8x512
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2DTranspose(256, kernel_size=5, strides=2, padding='same', use_bias=False)(x)  # 16x16x256
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2DTranspose(128, kernel_size=5, strides=2, padding='same', use_bias=False)(x)  # 32x32x128
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2DTranspose(3, kernel_size=5, strides=2, padding='same', use_bias=False, activation='tanh')(x)  # 64x64x3

    return Model(inputs=noise_input, outputs=x, name="Generator")

def build_DiscriminatorModel():
    image_input = layers.Input(shape=(64, 64, 3))

    x = layers.Conv2D(64, kernel_size=5, strides=2, padding='same')(image_input)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(128, kernel_size=5, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(256, kernel_size=5, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(512, kernel_size=5, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(1, activation='sigmoid')(x)

    return Model(inputs=image_input, outputs=x, name="Discriminator")

GeneratorModel, DiscriminatorModel = build_GeneratorModel(), build_DiscriminatorModel()
NOISE_DIM = GeneratorModel.input_shape
print(GeneratorModel.summary())
print(DiscriminatorModel.summary())

### Training

In [ ]:
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)  # car output= sigmoid

@tf.function
def train_step(real_images, Generator, Discriminator, batch_size):
    # === Génération d'images fausses ===
    noise = tf.random.normal([batch_size, NOISE_DIM])
    fake_images = Generator(noise, training=True)

    # === Discriminateur ===
    real_labels = tf.ones((batch_size, 1))
    fake_labels = tf.zeros((batch_size, 1))

    with tf.GradientTape() as disc_tape:
        real_output = Discriminator(real_images, training=True)
        fake_output = Discriminator(fake_images, training=True)

        disc_loss_real = loss_fn(real_labels, real_output)
        disc_loss_fake = loss_fn(fake_labels, fake_output)
        disc_loss = disc_loss_real + disc_loss_fake

    grads_disc = disc_tape.gradient(disc_loss, Discriminator.trainable_variables)
    OPTIMIZER.apply_gradients(zip(grads_disc, Discriminator.trainable_variables))

    # === Générateur ===
    noise = tf.random.normal([batch_size, NOISE_DIM])
    with tf.GradientTape() as gen_tape:
        generated_images = Generator(noise, training=True)
        fake_output = Discriminator(generated_images, training=True)
        gen_loss = loss_fn(real_labels, fake_output)  # veut tromper le disc

    grads_gen = gen_tape.gradient(gen_loss, Generator.trainable_variables)
    OPTIMIZER.apply_gradients(zip(grads_gen, Generator.trainable_variables))

    return gen_loss, disc_loss

best_val_loss = float("inf")
wait = 0
for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    # --- Entraînement ---
    progress_bar = tqdm(TrainGen, desc="Training", leave=False)
    for real_images in progress_bar:
        g_loss, d_loss = train_step(real_images, GeneratorModel, DiscriminatorModel, BATCH_SIZE)
        progress_bar.set_postfix({"g_loss": f"{g_loss:.4f}", "d_loss": f"{d_loss:.4f}"})

    # --- Validation ---
    val_g_losses = []
    for val_real_images in ValGen:
        noise = tf.random.normal([val_real_images.shape[0], NOISE_DIM])
        generated_images = GeneratorModel(noise, training=False)
        val_fake_output = DiscriminatorModel(generated_images, training=False)
        val_loss = loss_fn(tf.ones_like(val_fake_output), val_fake_output)
        val_g_losses.append(val_loss)

    avg_val_g_loss = tf.reduce_mean(val_g_losses)
    print(f"Validation Generator Loss: {avg_val_g_loss:.4f}")

    # --- Early Stopping ---
    if avg_val_g_loss < best_val_loss:
        best_val_loss = avg_val_g_loss
        wait = 0
        # Optionnel : sauvegarde du meilleur modèle
        GeneratorModel.save_weights("best_generator_weights.h5")
        DiscriminatorModel.save_weights("best_discriminator_weights.h5")
    else:
        wait += 1
        if wait >= PATIENCE:
            print(f"Early stopping at epoch {epoch+1} (no improvement in {PATIENCE} epochs).")
            break

